## **1. Install and import libraries**

In [1]:
!pip install -qq datasets evaluate transformers[sentencepiece]
!pip install -qq accelerate
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

In [3]:
notebook_login()

## **2. Download and load poem dataset**

In [ ]:
!unzip poem_dataset_final.zip

In [4]:
DATASET_PATH = 'poem_final.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [ ]:
print(df['content'][0])

In [6]:
def split_content(content):
    samples = []

    ### YOUR CODE START HERE
    poem_parts = content.split('\n\n')
    for poem_part in poem_parts:
      poem_in_lines = poem_part.split('\n')
      if len(poem_in_lines) > 4:
        samples.append(poem_in_lines)
    ### YOUR CODE END HERE

    return samples

df['content'] = df['content'].apply(lambda x: split_content(x))
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,[],"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,[],"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,[],"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”",[],"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,[],"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,[],NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,[],"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,"[[Cây bút gẫy trong tay, Cặn mực khô đáy lọ, Á...",19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,[],NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [7]:
df_exploded = df.explode('content')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=['content'])
df_exploded

,Unnamed: 0,title,content,source,url
11,11,“Lặng ngồi chốn tĩnh lâu”,"[Lặng ngồi chốn tĩnh lâu, Trăng sáng, gió canh...","Nguồn: Niệm Phật thập yếu, Thích Thiền Tâm, NX...",https://www.thivien.net/Th%C3%ADch-Thi%E1%BB%8...
12,12,“Lòng tôi là cây khế...”,"[Lòng tôi là cây khế, Em là chim về chơi, Vàng...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
18,18,“Tâm kệ”,"[Ba hồi âm định lực, ngọn triều chuyển sóng ca...","Nguồn: Phạm Thiên Thư, Suối nguồn vi diệu, NXB...",https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
38,38,1,"[Cùng quây quần lại đây, Nào các em miền xuôi,...","Nguồn: Vua Rồng xứ Lạc (truyện thơ), Nguyễn Ba...",https://www.thivien.net/Nguy%E1%BB%85n-Bao/1/p...
39,38,1,"[Có em nào biết được, Vì sao trên đất này, Rừn...","Nguồn: Vua Rồng xứ Lạc (truyện thơ), Nguyễn Ba...",https://www.thivien.net/Nguy%E1%BB%85n-Bao/1/p...
...,...,...,...,...,...
217,94,Ám ảnh,"[Cái giếng sâu thăm thẳm, Cái giếng sâu kinh k...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
220,97,Anh,"[Cây bút gẫy trong tay, Cặn mực khô đáy lọ, Án...",19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
221,97,Anh,"[Thôi đừng buồn nữa anh, Tấm rèm cửa màu xanh,...",19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
222,97,Anh,"[Ở ngoài kia trời gió, Ở ngoài kia trời mưa, C...",19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...


In [8]:
df_exploded['content'] = df_exploded['content'].apply(lambda x: '\n'.join(x))
df_exploded

,Unnamed: 0,title,content,source,url
11,11,“Lặng ngồi chốn tĩnh lâu”,"Lặng ngồi chốn tĩnh lâu\nTrăng sáng, gió canh ...","Nguồn: Niệm Phật thập yếu, Thích Thiền Tâm, NX...",https://www.thivien.net/Th%C3%ADch-Thi%E1%BB%8...
12,12,“Lòng tôi là cây khế...”,Lòng tôi là cây khế\nEm là chim về chơi\nVàng ...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
18,18,“Tâm kệ”,Ba hồi âm định lực\nngọn triều chuyển sóng cao...,"Nguồn: Phạm Thiên Thư, Suối nguồn vi diệu, NXB...",https://www.thivien.net/Ph%E1%BA%A1m-Thi%C3%AA...
38,38,1,Cùng quây quần lại đây\nNào các em miền xuôi\n...,"Nguồn: Vua Rồng xứ Lạc (truyện thơ), Nguyễn Ba...",https://www.thivien.net/Nguy%E1%BB%85n-Bao/1/p...
39,38,1,Có em nào biết được\nVì sao trên đất này\nRừng...,"Nguồn: Vua Rồng xứ Lạc (truyện thơ), Nguyễn Ba...",https://www.thivien.net/Nguy%E1%BB%85n-Bao/1/p...
...,...,...,...,...,...
217,94,Ám ảnh,Cái giếng sâu thăm thẳm\nCái giếng sâu kinh kh...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
220,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
221,97,Anh,Thôi đừng buồn nữa anh\nTấm rèm cửa màu xanh\n...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
222,97,Anh,Ở ngoài kia trời gió\nỞ ngoài kia trời mưa\nCâ...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...


In [9]:
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset

Dataset({
    features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
    num_rows: 65
})

In [10]:
TEST_SIZE = 0.1
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 58
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 7
    })
})

## **3. Preprocess dataset**

In [11]:
MODEL_NAME = 'danghuy1999/gpt2-viwiki'

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

In [12]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function(row):
    ### YOUR CODE START HERE

    return tokenizer(
        row["content"],
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True
    )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset["train"].column_names
)

### YOUR CODE END HERE

Map (num_proc=4):   0%|          | 0/58 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/7 [00:00<?, ? examples/s]

In [13]:
tokenizer('xin chào các bạn', max_length=10, padding='max_length', truncation=True)

{'input_ids': [6504, 3939, 331, 1486, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [14]:
tokenized_poem_dataset['train'][0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [15]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## **4. Training**

In [16]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [18]:
### YOUR CODE START HERE
training_args = TrainingArguments(
    output_dir="gpt2_viet_poem_generation",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True
)
### YOUR CODE END HERE

In [19]:
### YOUR CODE START HERE
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset["train"],
    eval_dataset=tokenized_poem_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)
### YOUR CODE END HERE
trainer.train()

<ipython-input-19-ad023a48b19a>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: npqace (npqace-university-of-greenwich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=80, training_loss=6.359362030029297, metrics={'train_runtime': 293.6075, 'train_samples_per_second': 1.975, 'train_steps_per_second': 0.272, 'total_flos': 29599488000000.0, 'train_loss': 6.359362030029297, 'epoch': 10.0})

In [20]:
trainer.push_to_hub(commit_message="Training complete")

events.out.tfevents.1740827108.a91b5b3edb17.540.0:   0%|          | 0.00/5.80k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/npqace/gpt2_viet_poem_generation/commit/2fb5348163f54d0e034d012485a94185c8401225', commit_message='Training complete', commit_description='', oid='2fb5348163f54d0e034d012485a94185c8401225', pr_url=None, repo_url=RepoUrl('https://huggingface.co/npqace/gpt2_viet_poem_generation', endpoint='https://huggingface.co', repo_type='model', repo_id='npqace/gpt2_viet_poem_generation'), pr_revision=None, pr_num=None)

## **5. Inference**

In [36]:
from transformers import pipeline

### YOUR CODE START HERE
prompt = "Học học nữa học mãi\n"
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
### YOUR CODE END HERE
results = results[0]
print()
for line in results.split('\n'):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Học học nữa học mãi
Trong vòng một giờ, người bạn vẫn đọc sách
Một ngày để đến từ đâu
Ngoài ra, khi chưa ai quên lời
Heo qua đường đi (về đêm, tôi chẳng chẳng)
Thành lâu dài lâu dài với tay, anh
